This notebook is associated with the paper "The relative class number one problem for function fields, III" by K.S. Kedlaya. It runs in SageMath (tested using version 9.5) and depends on Magma (tested using version
2.26-9).

In this notebook, we identify trigonal curves of genus 6 and Maroni invariant 2 which are candidates for the curve $C$ in a purely geometric quadratic extension $F'/F$ of relative class number 1. Allow 10 minutes for completion.

In [1]:
load("preamble.sage")

Construct the sets of $\mathbb{F}_{2^i}$-rational points of a specific $(2,1)$-hypersurface $X_1$ in $\mathbf{P}^1 \times \mathbf{P}^2$ for $i=1,2,3,4$.

In [2]:
F = GF(2)
P.<x0,x1,y0,y1,y2> = F[]
gen1 = x0^2*y0 + x0*x1*y1 + x1^2*y2

In [3]:
S1 = [vector(t) for t in ProjectiveSpace(F, 1)]
S2 = [vector(t) for t in ProjectiveSpace(F, 2)]
for v in S1 + S2:
    v.set_immutable()
S0 = list(itertools.product(S1, S2))
S = [x for x in S0 if gen1(*x[0], *x[1]) == 0]
len(S)

9

In [4]:
F4 = GF(4)
S14 = [vector(t) for t in ProjectiveSpace(F4, 1)]
S24 = [vector(t) for t in ProjectiveSpace(F4, 2)]
for v in S14 + S24:
    v.set_immutable()
S04 = list(itertools.product(S14, S24))
S4 = [x for x in S04 if gen1(*x[0], *x[1]) == 0]
len(S4)

25

In [5]:
F8 = GF(8)
S18 = [vector(t) for t in ProjectiveSpace(F8, 1)]
S28 = [vector(t) for t in ProjectiveSpace(F8, 2)]
for v in S18 + S28:
    v.set_immutable()
S08 = list(itertools.product(S18, S28))
S8 = [x for x in S08 if gen1(*x[0], *x[1]) == 0]
len(S8)

81

In [6]:
F16 = GF(16)
S116 = [vector(t) for t in ProjectiveSpace(F16, 1)]
S216 = [vector(t) for t in ProjectiveSpace(F16, 2)]
for v in S116 + S216:
    v.set_immutable()
S016 = list(itertools.product(S116, S216))
S16 = [x for x in S016 if gen1(*x[0], *x[1]) == 0]
len(S16)

289

Construct a subgroup $G$ of $\mathrm{GL}(2, \mathbb{F}_2) \times \mathrm{GL}(3, \mathbb{F}_2)$ fixing $X_1$.

In [7]:
l0 = [Matrix(F,[[1,1,0,0,0],[0,1,0,0,0],[0,0,1,0,0],[0,0,0,1,0],[0,0,1,1,1]]),
      Matrix(F,[[0,1,0,0,0],[1,0,0,0,0],[0,0,0,0,1],[0,0,0,1,0],[0,0,1,0,0]])]
G0 = GL(5,F).subgroup(l0)
G0.order()

6

Use an orbit lookup tree to find $G$-orbit representatives for 6-tuples of $\mathbb{F}_2$-points in $X_1$.

In [8]:
def apply_group_elem(g, x):
    g1 = g.submatrix(nrows=2,ncols=2)
    g2 = g.submatrix(row=2,col=2)
    v1 = g1*x[0]
    v2 = g2*x[1]
    v1.set_immutable()
    v2.set_immutable()
    return (v1, v2)

In [9]:
assert all(apply_group_elem(g, x) in S for g in l0[:1] for x in S)

In [10]:
def stabilizer(x):
    G1 = vec_stab(Matrix(F, x[0]), transpose=True)
    G2 = vec_stab(Matrix(F, x[1]), transpose=True)
    l0 = [block_matrix(2,2,[g.matrix(),0,0,identity_matrix(3)], subdivide=False) for g in G1.gens()] + \
        [block_matrix(2,2,[identity_matrix(2),0,0,g.matrix()], subdivide=False) for g in G2.gens()]
    return GL(5, F).subgroup(l0)

In [11]:
def optimized_rep(g):
    return g.matrix()

In [12]:
methods = {'apply_group_elem': apply_group_elem,
           'stabilizer': stabilizer,
           'optimized_rep': optimized_rep}
tree = build_orbit_tree(G0, S, 6, methods, verbose=False)

In [13]:
[len(green_nodes(tree, s)) for s in range(4, 7)]

[24, 24, 17]

For $k=4,5,6$, for each $k$-tuple of $\mathbb{F}_2$-rational points of $X_1$, find $(1,3)$-surfaces $X_2$ such that $X_1 \cap X_2$ contains precisely the chosen set of $\mathbb{F}_2$-points. We also enforce the desired point counts over $\mathbb{F}_{2^i}$ for $i=2,3,4$.

In [14]:
monos13 = [prod(x) for x in itertools.product([prod(y) for y in itertools.combinations_with_replacement([x0,x1],1)],
                                              [prod(y) for y in itertools.combinations_with_replacement([y0,y1,y2],3)])]
len(monos13)

20

In [15]:
coords13 = {x: vector(F, (mu(*x[0], *x[1]) for mu in monos13)) for x in S}

In [16]:
perp = Matrix([coords13[x] for x in S])
perp.set_immutable()
tmp2 = set(t[:2] for t in targets6)
tmp3 = set(t[:3] for t in targets6)
tmp4 = set(t[:4] for t in targets6)
curves = defaultdict(list)
for s in range(4,7):
    for vecs in green_nodes(tree, s):
        target = vector(F, (0 if x in vecs else 1 for x in S))
        for v2 in solve_right_iterator(perp, target):
            gen2 = sum(v2[i]*monos13[i] for i in range(20))
            s2 = sum(1 for x in S4 if gen2(*x[0], *x[1]) == 0)
            if (s,s2) in tmp2:
                s3 = sum(1 for x in S8 if gen2(*x[0], *x[1]) == 0)
                if (s,s2,s3) in tmp3:
                    s4 = sum(1 for x in S16 if gen2(*x[0], *x[1]) == 0)
                    if (s,s2,s3,s4) in tmp4:
                        curves[(s,s2,s3,s4)].append((gen1, gen2))

In [17]:
[(s, len(curves[s])) for s in curves]

[((4, 14, 16, 18), 28),
 ((5, 11, 11, 31), 497),
 ((5, 13, 14, 25), 124),
 ((5, 15, 5, 35), 4),
 ((5, 11, 11, 39), 32),
 ((6, 14, 12, 26), 6),
 ((6, 10, 9, 38), 18),
 ((6, 14, 6, 34), 6),
 ((6, 14, 6, 26), 6)]

Close out this case.

In [18]:
I1 = P.ideal([x0,x1])
I2 = P.ideal([y0,y1,y2])
CR = magma.CoxRing(P, [I1,I2], [[1,1,0,0,0],[0,0,1,1,1]], [])
proj = CR.ToricVariety()

In [19]:
closeout(curves, X=proj, genus=6)

Number of curves found: 64
Number of isomorphism classes found: 9
No covers found in this case!
Total time: 9 minutes
